In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

#Import Libraries
import tensorflow as tf
import numpy as np
import math
import os
import matplotlib
import matplotlib.pyplot as plt
import datetime
import time

import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from models import save_model, generate_model_CNN, generate_model_LSTM, plot_performance
from generators import generator, decode

Using TensorFlow backend.


In [2]:
save_dir = 'plots'
if not os.path.exists(save_dir):
        os.makedirs(save_dir)

X_train = np.load('../data/final_data/X_train.npy')
y_train = np.load('../data/final_data/y_train.npy')


X_test = np.load('../data/final_data/X_test.npy')
y_test = np.load('../data/final_data/y_test.npy')


X = np.load('../data/final_data/X.npy')
y = np.load('../data/final_data/y.npy')

enc = OneHotEncoder(categories='auto')
y = enc.fit_transform(y.reshape(-1, 1) ,).toarray()
y_train = enc.fit_transform(y_train.reshape(-1, 1) ,).toarray()
y_test = enc.fit_transform(y_test.reshape(-1, 1) ,).toarray()


FileNotFoundError: [Errno 2] No such file or directory: '../data/final_data/X_train.npy'

In [ ]:
batch_size = 500
time_window = X_test.shape[1]
n_dim = X_test.shape[2]
predict_early = 0

input_shape = (time_window, n_dim)
class_weight = {0: 1., 1: 5.,}

In [ ]:
input_shape = (time_window, n_dim)
model = generate_model_CNN(input_shape)

# input_shape = (None, n_dim)
# model = generate_model_LSTM(input_shape)

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=50,
                    epochs=25, 
                    verbose=1,
                    validation_data = (X_test, y_test),
                    class_weight=class_weight,
                    )

save_model(model, name=predict_early)

In [ ]:
plot_performance(history)

In [ ]:
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

from generators import decode

In [ ]:
res = model.predict(X_test) 

threashold = 0.5
y_pred = decode(res, threshold=threashold)
y_true = decode(y_test, threshold=threashold)

In [ ]:
mask = y_true==1
bins = 100

plt.figure()
plt.hist(res[mask,1], bins=bins, histtype='step', density=True, cumulative=1)
plt.hist(res[np.logical_not(mask),1], bins=bins, histtype='step', density=True, cumulative=-1)
plt.show()

plt.figure()
plt.hist(res[mask,1], bins=bins, histtype='step', density=False, cumulative=1)
plt.hist(res[np.logical_not(mask),1], bins=bins, histtype='step', density=False, cumulative=-1)
plt.show()

In [ ]:



print('precision_score: ', np.round(precision_score(y_true, y_pred, average='binary', pos_label=1), decimals=3))
print('recall_score: ', np.round(recall_score(y_true, y_pred, average='binary', pos_label=1), decimals=3))
print('')
print('precision_score: ', np.round(precision_score(y_true, y_pred, average='binary', pos_label=0), decimals=3))
print('recall_score: ', np.round(recall_score(y_true, y_pred, average='binary', pos_label=0), decimals=3))

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print(cm)
print()
print(cm/cm.sum(axis=1)[:, np.newaxis])

In [ ]:
n_occ = [y_true.shape[0]-y_true.sum(), y_true.sum()]

precision, recall, thresholds = precision_recall_curve(y_true, res[:,1])


plt.figure()
plt.plot(recall, precision)
plt.xlabel('recall')
plt.ylabel('precission')
plt.show()


In [ ]:
tp = np.logical_and(y_true==1, y_pred==1)
fp = np.logical_and(y_true==0, y_pred==1)
fn = np.logical_and(y_true==1, y_pred==0)

In [ ]:
tp.sum(), fp.sum(), fn.sum()

In [ ]:
mask = tp + fp + fn
X_expo = X_test[mask]

In [ ]:
import scipy.io as sio

In [ ]:
sio.savemat('test.mat', {'vect':np.array(X_expo)})

In [ ]:
>>> import numpy as np
>>> vect = np.arange(10)
>>> vect.shape
(10,)
>>> sio.savemat('np_vector.mat', {'vect':vect})